In [ ]:
import pandas as pd
import numpy as np
import pickle
import time
import math
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import re
import csv
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

pd.set_option('display.max_columns', 50)


# change this directory to a folder with all the files you want in it 
directory_path = '/home/jhj5dh/ARI/dspg22ari2/BERT_Analysis/GAT'

# list of file names in the directory
list_of_files = os.listdir(directory_path)

# initialize an empty dataframe to store the text documents
df = pd.DataFrame(columns=['text'])

for filename in list_of_files:
    file_path = os.path.join(directory_path, filename)
    # check if the path is a file
    if os.path.isfile(file_path):
        # read the contents of the file and append as a new row to the dataframe
        print(file_path)
        try:
            
            text = pd.read_csv(file_path, encoding='utf-8', quoting=csv.QUOTE_NONE, lineterminator='.', header=None)[0].str.cat().strip()
        except:
            print(filename+" didn't work")
            
        df = pd.concat([df, pd.DataFrame({'text': [text]})], ignore_index=True)


# this line is taking out the new lines so it doesn't display them all as separate documents
df = df.replace(r'\n',' ', regex=True)
df = df.replace('', ' ', regex=True)




In [ ]:
for filename in list_of_files:
    file_path = os.path.join(directory_path, filename)
    # check if the path is a file
    if os.path.isfile(file_path):
        with open(file_path) as f:
            data = f.readlines()
            
            print(type(data))
            

In [ ]:
len(df)

In [ ]:
df.text[0]

In [ ]:
import unicodedata
df['text'] = df['text'].apply(lambda x: ''.join([' ' if not unicodedata.normalize('NFKD', char).encode('ASCII', 'ignore') else char for char in x]))

model_name = 'gpt2'
gp2_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
df.shape

In [ ]:
test_vector=tokenizer("It is very much like me that I am good at changing myself to adjust to changes in my life It is very much like me that It is difficult for me to adjust to changes It is very much like me that I can usually fit myself into any situation")


In [ ]:
test_vector

In [ ]:
def encode_document(text):
    input_ids = tokenizer.encode(text, max_length=4, truncation=True, padding='max_length', return_tensors="pt")
    outputs = gp2_model(input_ids)
    last_hidden_state = outputs.logits
    document_embedding = torch.mean(last_hidden_state, dim=1).detach().numpy()
    return document_embedding

In [ ]:
vectors = df['text'].apply(lambda x: encode_document(str(x)))
output_vectors = pd.DataFrame(vectors.tolist())

# Write the output vectors to a csv file
# change the output vector name every time you change the folder
output_vectors.to_csv('output_vectors_FM_GP2.csv', index=False)

In [ ]:
output_vectors.shape

In [ ]:
output_vectors